# Genetic algorithm 

## 

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from operator import attrgetter

## 1.  read data in
- List of relative frequencies of techniques in X_small
- y_train to train the GA
- y_validation to later test the GA result

In [2]:
# read in excel with techniques and probabilities, sheet X removed latest 20%
X = pd.read_excel("../reduced_table_with_timestamps_and_props.xlsx", sheet_name="X", index_col="ID")

# extract probabilities from excel and save as pd data frame
probs = pd.DataFrame(X.iloc[0])[1:]
probs

,prop
T1548.002,0.00493189
T1548.001,0.000469704
T1134,0.00164396
T1134.002,0.00164396
T1134.004,0.000234852
...,...
T1102,0.000469704
T1102.002,0.00516674
T1102.001,0.00164396
T1102.003,0.000704556


In [3]:
# read in excel with newest 20 percent of techniques for evaluation
y_train = pd.read_excel("../reduced_table_with_timestamps_and_props.xlsx", sheet_name="y", index_col="ID")
y_train = y_train.drop(y_train.tail(2).index, axis=0)
y_train = y_train.drop(['created'], axis = 1).T
y_train


ID,S0461,S0465,S0462,S0464,S0466,S0467,S0468,S0469,S0470,S0471,...,S0593,S0592,S0594,S0595,S0596,S0597,S0600,S0599,S0601,S0598
T1548.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T1548.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T1134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T1134.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T1134.004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
T1102.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T1102.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T1102.003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# read correlation matrix
corr_mat = pd.read_excel("../regression/corr_techniques.xlsx", sheet_name="Sheet1")
corr_mat = corr_mat.drop('Unnamed: 0', 1)
corr_mat

,1,2,3,4,5,6,7,8,9,10,...,260,261,262,263,264,265,266,267,268,269
0,1.000000,-0.015059,0.103402,0.238980,0.270487,0.127714,0.328829,-0.015059,0.110425,0.056578,...,0.115306,0.025799,0.167711,0.014296,-0.015059,-0.032176,0.028190,0.094802,-0.021340,0.075569
1,-0.015059,1.000000,-0.008703,-0.008703,-0.004073,-0.004073,-0.008197,-0.004073,-0.011689,-0.007084,...,-0.014438,-0.009642,-0.016246,-0.010911,-0.004073,-0.008703,-0.014751,-0.009183,-0.005772,-0.020961
2,0.103402,-0.008703,1.000000,0.207759,-0.008703,0.229665,-0.017513,0.229665,0.146032,-0.015136,...,0.039559,-0.020600,0.028669,-0.023313,-0.008703,-0.018595,0.037540,-0.019621,-0.012333,0.108543
3,0.238980,-0.008703,0.207759,1.000000,0.229665,0.229665,0.222324,-0.008703,0.231536,-0.015136,...,0.109965,-0.020600,-0.034711,-0.023313,-0.008703,-0.018595,0.037540,-0.019621,-0.012333,0.108543
4,0.270487,-0.004073,-0.008703,0.229665,1.000000,-0.004073,0.244369,-0.004073,0.168394,-0.007084,...,-0.014438,-0.009642,-0.016246,-0.010911,-0.004073,-0.008703,-0.014751,-0.009183,-0.005772,0.086683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,-0.032176,-0.008703,-0.018595,-0.018595,-0.008703,-0.008703,-0.017513,-0.008703,0.060529,-0.015136,...,-0.030847,0.081985,-0.034711,0.067903,-0.008703,1.000000,-0.031517,-0.019621,-0.012333,0.108543
265,0.028190,-0.014751,0.037540,0.037540,-0.014751,0.130694,-0.029684,-0.014751,0.009842,0.058663,...,0.076596,0.027679,0.134530,-0.039513,-0.014751,-0.031517,1.000000,0.163488,-0.020904,0.017649
266,0.094802,-0.009183,-0.019621,-0.019621,-0.009183,-0.009183,-0.018480,-0.009183,-0.026353,0.115259,...,-0.032550,0.075684,0.143943,-0.024599,-0.009183,-0.019621,0.163488,1.000000,-0.013014,0.001280
267,-0.021340,-0.005772,-0.012333,-0.012333,-0.005772,-0.005772,-0.011616,-0.005772,-0.016564,-0.010039,...,-0.020459,0.139425,-0.023022,-0.015462,-0.005772,-0.012333,-0.020904,-0.013014,1.000000,0.046567


## 2. Create helper functions
- create_starting_population
- calculate_fitness
- select_individual_by_tournament
- select_individual_by_roulette
- breed_by_crossover_1point
- breed_by_crossover_2point
- breed_by_crossover_uniform
- randomly_mutate_population

In [5]:
# function to create a starting population by drawing zeros and ones randomly. Chance for ones is prob_of_ones
# individuals: int is number of individuals (=software) to be generated
# length_of_techniques: int
# percent_of_ones: float
def create_starting_population(individuals, length_of_techniques, prob_of_ones=0.04):
    #length_of_techniques = len(probs)
    size = (individuals, length_of_techniques)
    probability = np.array([ 1 - prob_of_ones, prob_of_ones])
    
    #np.random.seed(42)
    population = np.random.choice([0,1], size=size, p=probability)
    
    return population

In [6]:
#example starting population
#print (create_starting_population(6, len(probs)) )

In [7]:
# function to calculate fitness of every individual in the current population
def calculate_fitness(individuals, probs, corr_mat, lamda_balance = 0.2, bonus_factor = 135, penalty_factor = 12, print_out = False):
    
    len_pop = len(probs)
    
    individuals = np.array(individuals) # individuals = 6x269
    probs = np.array(probs) # array with probabilities
    ones = np.ones(len_pop) # number of ones 269x1
    
    occurrences = (individuals @ ones) # number of ones in individual 6x1
    P = individuals # 6x269
    
    corr = np.array(corr_mat).reshape(len_pop,len_pop)
    np.fill_diagonal(corr, 0) # 269x269, correlation matrix with 0 self covariance
    A = (P @ corr) # 6x269
    K = (P @ A.T) # 6x6
    corr_term = K.diagonal()/len_pop # diagonal elements of matrix K 

    
    prob_term = (individuals @ probs) # a includes values between 0 and 1
    difference = (np.subtract(occurrences,8)) # here random N(11, 6)

    pen_1 = (difference**2) * np.sign(difference)/penalty_factor
    pen_1[pen_1 < 0] = 0
    
    pen_2 = np.copy(difference)/bonus_factor
    pen_2[pen_2 > 0] = 0
    penalty_term = (pen_1 + pen_2)
        
    if print_out:
        print("prob_term")
        print(prob_term)
        print("")
        print(occurrences)
        print("difference")
        print(difference)
        print("penalty_term")
        print(penalty_term)
        print("corr_term")
        print(corr_term)
        print("len_pop")
        print(len_pop)
        print("")
        
    fitness_scores = ((lamda_balance * prob_term.T + corr_term * (1-lamda_balance)) - penalty_term)
    return fitness_scores[0]

In [8]:
#testPop = create_starting_population(6, len(probs))
#print('Startpopulation')
#print(testPop)

#calculate_fitness(testPop, probs, corr_mat, print_out = True)

In [9]:
#function to select individuals by using tournament selection
#random.seed(42)
def select_individual_by_tournament(population, scores):
    # get population size
    population_size = len(scores)

    # pick individuals for tournament
    #random.seed(42)
    fighter_1 = random.randint(0, population_size-1)
    print(fighter_1)
    #random.seed(2021)
    fighter_2 = random.randint(0, population_size-1)
    print(fighter_2)
    
    # get fitness score for each individual
    fighter_1_fitness = scores[fighter_1]
    fighter_2_fitness = scores[fighter_2]
    
    # identify undividual with highest fitness
    # fighter 1 will win if scores are equal
    if fighter_1_fitness >= fighter_2_fitness:
        winner = fighter_1
    else:
        winner = fighter_2
    
    #return the chromsome of the winner
    return population[winner, :]

In [10]:
#select_individual_by_tournament(testPop, calculate_fitness(testPop, probs, corr_mat))

In [11]:
# function to select individuals by using roulette wheel selection
# cannot be used yet, since it does not accept negative fitness values
# fitness value will represent the selection probability, which can't be negative
# idea1: normalise fitness values
# idea2: subtract the lowest (negative) value from all fitness values. The lowest fitness value is now zero.

#np.random.seed(42)
def select_individual_by_roulette(population, k): # list of individuals to select from, number of individuals to select, attribute to use as selection criteria
    scores = calculate_fitness(population, probs, corr_mat)
    scores_normalized = scores -min(scores)
    scores_list = []
    for i in range(len(population)):
        fitness_sc = scores_normalized[i]
        scores_list.append((i, fitness_sc))
    scores_df = pd.DataFrame(scores_list, columns =["Index", "Fitness"])

    
    scores_df["Fitness"] = scores_df["Fitness"]/scores_df["Fitness"].sum()
    
    chosen = np.random.choice(scores_df["Index"], size = k, p = scores_df["Fitness"].fillna(0))
    winner1 = population[chosen[0], :]
    winner2 = population[chosen[1], :]


    return winner1, winner2

In [12]:
# implementation of roulette wheel selection
# def select_individual_by_roulette(population, probabilities, number):
#     chosen = []
#     for n in range(number):
#         r = random.random()
#         for (i, individual) in enumerate(population):
#             if r <= probabilities[i]:
#                 chosen.append(list(individual))
#                 break
#     return chosen

In [13]:
# function for executing one point crossover
def breed_by_crossover_1point(ind1, ind2):
    chromosome_length = min(len(ind1), len(ind2))
    #random.seed(42)
    crossover_point = random.randint(1, chromosome_length-1)
    
    ind1[crossover_point:], ind2[crossover_point:] = ind2[crossover_point:], ind1[crossover_point:]
    
    return ind1, ind2

In [14]:
# function for executin two point crossover
def breed_by_crossover_2point(ind1, ind2):
    chromosome_length = min(len(ind1), len(ind2))
    #random.seed(1)
    crossover_point1 = random.randint(1, chromosome_length)
    #random.seed(2)
    crossover_point2 = random.randint(1, chromosome_length-1)
    
    if crossover_point2 >= crossover_point1:
        crossover_point2 += 1
    else: # swapping the two crossover points
        crossover_point1, crossover_point2 = crossover_point2, crossover_point1
        
    ind1[crossover_point1: crossover_point2], ind2[crossover_point1: crossover_point2] \
        = ind2[crossover_point1: crossover_point2], ind1[crossover_point1: crossover_point2]
    
    return ind1, ind2

In [15]:
# function for executing uniform crossover
#random.seed(242)
def breed_by_crossover_uniform(ind1, ind2, indpb):
    # parameter indpb is the indipendent probability for each bit to be exchanged
    # get length of chromosome
    chromosome_length = min(len(ind1), len(ind2))
    
    for i in range(chromosome_length):
        #random.seed(42)
        if random.random() < indpb: #add: lower probability that 0 --> 1 than the probability that 1 --> 0
             ind1[i], ind2[i] = ind2[i], ind1[i]
    return ind1, ind2

In [16]:
# function to mutate population
#np.random.seed(123)
def randomly_mutate_population(population, mutation_probability):
    
    # apply random mutation
    #np.random.seed(123)
    random_mutation_array = np.random.random(
        size=(population.shape))

    random_mutation_boolean = \
        random_mutation_array <= mutation_probability

    population[random_mutation_boolean] = \
    np.logical_not(population[random_mutation_boolean])

    # return mutation population
    return population

In [17]:
import operator
# class to combine scores, occurrences and population
class result:
    def __init__(self, score, occurences, population):
        self.score = score
        self.occurences = occurences
        self.population = population

## 3. Run Genetic Algorithm

In [37]:
# main algorithm code
# set general parameters

# toDo, choose cross alogrithm
def run_ga(probs, corr_mat, population_size = 20, maximum_generation = 300, mutation_rate =0.001, fix_seed=False, print_out=False):
    if fix_seed:
        np.random.seed(42)
        random.seed(42)
    
    chromosome_length = len(probs)
    best_score_progress = [] # tracks progress

    # create starting population
    population = create_starting_population(population_size, chromosome_length)

    # print (population)
    # display best score in starting population
    scores = calculate_fitness(population, probs, corr_mat)
    best_score = np.max(scores)

    if print_out:
        print ('Starting best score: %.1f' %best_score)

    # add starting best score to progress tracker
    best_score_progress.append(best_score)

    # going through the generations of genetic algorithm
    for generation in range(maximum_generation):
        # create an empty list for new population
        new_population = []

        # create new popualtion generating two children at a time
        for i in range(int(population_size/2)):
            #parent_1 = select_individual_by_tournament(population, scores)
            #parent_2 = select_individual_by_tournament(population, scores)
            parent1, parent2 = select_individual_by_roulette(population, 2)
            #child_1, child_2 = breed_by_crossover_1point(parent_1, parent_2)
            #child_1, child_2 = breed_by_crossover_2point(parent_1, parent_2) # Results are not as promising as with the others
            child_1, child_2 = breed_by_crossover_uniform(parent1, parent2, 0.5)
            new_population.append(child_1)
            new_population.append(child_2)

        # replace the old population with the new one
        population = np.array(new_population)

        # apply mutation
        population = randomly_mutate_population(population, mutation_rate)

        # score best solution, and add to tracker
        scores = calculate_fitness(population, probs, corr_mat)
        best_score = np.max(scores)
        best_score_progress.append(best_score)

        # print(best_score)

    # GA has completed required generation number

    ones = np.ones(len(probs)) # number of ones 269x1
    occurrences = (population @ ones)
    if print_out:
        print ('End best score %.1f' %best_score)
        print (occurrences)

        # plot progress
        %matplotlib inline
        plt.plot(best_score_progress)
        plt.xlabel('Generation')
        plt.ylabel('Best score (% target)')
        plt.show()
        
        # Print out information about best individuum
        print('Best score target generation: %.1f' %best_score)
        
    max_index_row = np.argmax(scores, axis=0)
    
    if print_out:
        print ('Position in Array:', max_index_row)
        print("Number of Ones:", occurrences[max_index_row])
        print("")
        print("Respective Chromosome:", population[max_index_row] )
        print("")

        print("Techniques used:")
        i = 0
        for row in probs.index: 
            if population [max_index_row][i] == 1:
                print(row)
            i+=1

    best_sw = np.tile(population[max_index_row],1) 
       
    
    
    # sort attribute scores in order to select best X individuums for evaluation
    results = []
    for i in range(0,len(scores)):
    #    print(i)
        r1 = result(scores[i], occurrences[i], population[i])
        results.append(r1)
 

    sorted_results = sorted(results, key=operator.attrgetter('score'))
    predi = sorted_results[-1]
    
    #print("predi")
    #print(predi)
    #print(type(predi))
    
    
    #return sorted_results, best_sw
    F1, softwareIndex, generatedSoftware = calculate_F_score(y_train, predi.population, do_print=False)
    return F1, softwareIndex, generatedSoftware

## 4. evaluate result of GA

In [19]:
# read in the y_train data set and the predicted software

def calculate_F_score(y_train: pd.DataFrame, predicted_sw: np.array, do_print=False):    
    #print(type(y_train), type(predicted_sw))
    assert y_train.shape[0] == predicted_sw.shape[0]
    
    ones = np.ones(len(y_train))

    occurences = np.array(predicted_sw).T @ ones
    
    F1_list = []
    Software_index = 0
    
    for software in y_train.columns:
        sum_s = y_train[software] @ predicted_sw

        precision = 0
        if occurences != 0:
            precision = sum_s/occurences
        
        number_of_techniques_y = y_train[software] @ ones
        recall = 0
        if number_of_techniques_y != 0:
            recall = sum_s/(number_of_techniques_y)
        F1 = 0
        if (precision + recall) != 0:
            F1 = 2 * precision* recall /(precision + recall)
            
        F1_list.append(F1)
    
    return max(F1_list), F1_list.index(max(F1_list)) ,predicted_sw

In [20]:
#calculate_F_score(y_train, sorted_results[0].population)

In [21]:
#sorted_results, best_sw = run_ga(probs, corr_mat, print_out=True)
#run_ga(probs, corr_mat, print_out=True)

## 5. Helper functions optimizer

In [22]:
lamda_balance = 0.2 # 0 - 0.5
bonus_factor = 135 # +-20
penalty_factor = 12 #+- 5
population_size = 20 # ggf in kleinem Rahmen bewegen lassen
maximum_generation = 400, # würde ich fix setzen
mutation_rate =0.001 # to optimise
prob_of_ones=0.04 # mean of ones in X_small

startValues = [lamda_balance, bonus_factor, penalty_factor, mutation_rate, prob_of_ones]
startValues

[0.2, 135, 12, 0.001, 0.04]

In [23]:
lowerBound = [0.01, 115, 7, 0.0008, 0.03]
upperBound = [0.99, 155, 12, 0.0012, 0.05]

bounds = tuple((lowerBound[x], upperBound[x]) for x in range(0,len(startValues)))
bounds

((0.01, 0.99), (115, 155), (7, 12), (0.0008, 0.0012), (0.03, 0.05))

In [31]:
def GA_func(params):
    assert len(params) == 5
    lamda_balance = params[0]
    bonus_factor = params[1]
    penalty_factor = params[2]
    mutation_rate = params[3]
    prob_of_ones = params[4]
    # ein bester F1 score optimieren ist noch nicht finales Ziel
    F1, indexOfF1, generated_software = run_ga(probs, corr_mat, population_size = 20, maximum_generation = 30, mutation_rate =mutation_rate, fix_seed=True  print_out=False)
    # gib hier noch malus, wenn immer der gleiche Index
    return -F1

In [32]:
GA_func(startValues)

-0.3333333333333333

## 6. Optimize hyper-parameter of model
#### 6.1 local optimizer "L-BFGS-B"

In [33]:
import scipy.optimize as sco
minimize = sco.minimize(GA_func, x0=startValues, bounds=bounds, method="L-BFGS-B", options={"gtol": 1e-12, "ftol":1e-12})
minimize.x

array([2.00e-01, 1.35e+02, 1.20e+01, 1.00e-03, 4.00e-02])

#### 6.2 global optimizer

In [34]:
dual_annealing = sco.dual_annealing(GA_func, x0=startValues, bounds=bounds, seed=19937, maxiter=20)

In [35]:
dual_annealing.x

array([7.68871328e-01, 1.54069943e+02, 1.10699430e+01, 1.07136582e-03,
       4.99429800e-02])

In [36]:
GA_func(dual_annealing.x)

-0.5

## 7.  Save series of generated softwares

In [ ]:
examples_to_save = 5
df_to_save = pd.DataFrame([])
F1_values = pd.DataFrame([])
maxIndexValues = pd.DataFrame([])
for i in range(examples_to_save): #population_size = 20, maximum_generation = 300, mutation_rate =0.001
    F1, maxIndex, generated_software = run_ga(probs, corr_mat, )
    
    df_to_save[f"software_{i}"] = generated_software
    print(F1)
    F1_values[f"software_{i}"] = np.array([F1])
    maxIndexValues[f"software_{i}"] = np.array([maxIndex])

maxIndexValues

In [ ]:
F1_values

In [ ]:
df_to_save.append(F1_values).append(maxIndexValues).to_excel('output3.xlsx', engine='xlsxwriter')